In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


from biotorch.benchmark.run import Benchmark
from biotorch.module.biomodule import BioModule

In [18]:
n_epochs = 100
batch_size_train = 16
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

Feedback Alignment on MNIST

In [19]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=True, 
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, 
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

print("Train size: ", len(train_loader.dataset))
print("Test size: ", len(test_loader.dataset))

Train size:  60000
Test size:  10000


### ANN implememnted with DFA and FA

In [44]:
# create an ANN model using pytorch of 3 layers with
# 1. 784 input neurons
# 2. 50 hidden neurons
# 3. 10 output neurons

class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(784, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [45]:
# covert the ANN model to a BioModule model

model = ANN()
fa = BioModule(model, mode='fa', output_dim=10)
print(fa)

Module has been converted to fa mode:

The layer configuration was:  {'type': 'fa', 'options': {'constrain_weights': False, 'gradient_clip': False, 'init': 'xavier'}}
- All the 2 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.
BioModule(
  (module): ANN(
    (fc1): Linear(in_features=784, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=10, bias=True)
  )
)


In [ ]:
from biotorch.training.trainer import Trainer
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import StepLR
from torch.optim import Adam

adam = Adam(fa.parameters(), lr=0.01)

lr_ = StepLR(optim.Adam(fa.parameters(), lr=0.01), step_size=0.1)
loss = CrossEntropyLoss()
metric_config = {
    'top_k': 5,
    'display_iterations': 100,  # Display metrics every 100 iterations
    'layer_alignment': False,     # Enable layer-wise alignment
    'weight_ratio': False      # Set weight ratio for alignment regularization
}


trainer = Trainer(model=fa, mode='dfa', loss_function=loss, optimizer=adam, train_dataloader=train_loader, val_dataloader=test_loader, device='cpu', epochs=3, output_dir='./output', lr_scheduler=lr_, metrics_config=metric_config)




AttributeError: 'SGD' object has no attribute 'Adam'

In [59]:
trainer.run()

Epoch: [0][  0/938]	Time  0.031 ( 0.031)	Data  0.012 ( 0.012)	Loss nan (nan)	Acc@1   4.69 (  4.69)	Acc@5  50.00 ( 50.00)
Epoch: [0][100/938]	Time  0.015 ( 0.025)	Data  0.005 ( 0.006)	Loss nan (nan)	Acc@1  10.94 (  9.38)	Acc@5  46.88 ( 49.77)
Epoch: [0][200/938]	Time  0.017 ( 0.024)	Data  0.004 ( 0.006)	Loss nan (nan)	Acc@1   3.12 (  9.40)	Acc@5  57.81 ( 50.07)
Epoch: [0][300/938]	Time  0.025 ( 0.023)	Data  0.005 ( 0.005)	Loss nan (nan)	Acc@1   7.81 (  9.62)	Acc@5  56.25 ( 50.30)
Epoch: [0][400/938]	Time  0.021 ( 0.023)	Data  0.006 ( 0.006)	Loss nan (nan)	Acc@1   6.25 (  9.62)	Acc@5  53.12 ( 50.60)
Epoch: [0][500/938]	Time  0.028 ( 0.024)	Data  0.005 ( 0.006)	Loss nan (nan)	Acc@1  14.06 (  9.62)	Acc@5  46.88 ( 50.51)
Epoch: [0][600/938]	Time  0.018 ( 0.024)	Data  0.005 ( 0.006)	Loss nan (nan)	Acc@1  14.06 (  9.63)	Acc@5  45.31 ( 50.69)
Epoch: [0][700/938]	Time  0.017 ( 0.024)	Data  0.006 ( 0.006)	Loss nan (nan)	Acc@1   6.25 (  9.79)	Acc@5  39.06 ( 50.72)
Epoch: [0][800/938]	Time  0.019 

KeyboardInterrupt: 

In [1]:
# resize MNIST images to 32x32
transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=True, download=True,
                             transform=transform),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data', train=False, download=True,
                                 transform=transform),
    batch_size=batch_size_test, shuffle=True)



NameError: name 'transforms' is not defined

In [62]:
# create a lenet model using pytorch

class LeNet(nn.Module):
    # def __init__(self):
    #     super(LeNet, self).__init__()
    #     self.conv1 = nn.Conv2d(1, 20, 5, 1)
    #     self.conv2 = nn.Conv2d(20, 50, 5, 1)
    #     self.fc1 = nn.Linear(4*4*50, 500)
    #     self.fc2 = nn.Linear(500, 10)

    # def forward(self, x):
    #     x = F.relu(self.conv1(x))
    #     x = F.max_pool2d(x, 2, 2)
    #     x = F.relu(self.conv2(x))
    #     x = F.max_pool2d(x, 2, 2)
    #     x = x.view(-1, 4*4*50)
    #     x = F.relu(self.fc1(x))
    #     x = self.fc2(x)
    #     return F.log_softmax(x, dim=1)
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=4, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=4, stride=1)
        self.relu3 = nn.ReLU()
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = self.relu3(out)
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.relu4(out)
        out = self.fc2(out)
        return out
    
lenet = LeNet()

# convert the LeNet model to a BioModule model

# model = LeNet()
alex = BioModule(lenet, mode='dfa', output_dim=10)
print(alex)
# adam = Adam(alex.parameters(), lr=0.01)
from torch.optim import SGD
optim = SGD(alex.parameters(), lr=0.01, momentum=0.9, weight_decay=10e-3)


lr_ = StepLR(adam, step_size=0.1)
loss = CrossEntropyLoss()
metric_config = {
    'top_k': 5,
    'display_iterations': 100,  # Display metrics every 100 iterations
    'layer_alignment': False,     # Enable layer-wise alignment
    'weight_ratio': False      # Set weight ratio for alignment regularization
}


trainer = Trainer(model=alex, mode='dfa', loss_function=loss, optimizer=optim, train_dataloader=train_loader, val_dataloader=test_loader, device='cpu', epochs=100, output_dir='./output', lr_scheduler=lr_, metrics_config=metric_config)


Module has been converted to dfa mode:

The layer configuration was:  {'type': 'dfa', 'options': {'constrain_weights': False, 'init': 'xavier', 'gradient_clip': False}}
- All the 3 <class 'torch.nn.modules.conv.Conv2d'> layers were converted successfully.
- All the 2 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.
BioModule(
  (module): LeNet(
    (conv1): Conv2d(1, 6, kernel_size=(4, 4), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1))
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(16, 120, kernel_size=(4, 4), stride=(1, 1))
    (relu3): ReLU()
    (fc1): Linear(in_features=120, out_features=84, bias=True)
    (relu4): ReLU()
    (fc2): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [63]:
trainer.run()

Epoch: [0][  0/938]	Time  0.036 ( 0.036)	Data  0.013 ( 0.013)	Loss 2.3121e+00 (2.3121e+00)	Acc@1   7.81 (  7.81)	Acc@5  42.19 ( 42.19)
Epoch: [0][100/938]	Time  0.039 ( 0.027)	Data  0.005 ( 0.006)	Loss 2.2642e+27 (1.3521e+26)	Acc@1  14.06 ( 10.26)	Acc@5  56.25 ( 50.74)
Epoch: [0][200/938]	Time  0.018 ( 0.025)	Data  0.006 ( 0.006)	Loss nan (nan)	Acc@1   4.69 ( 10.11)	Acc@5  46.88 ( 50.52)
Epoch: [0][300/938]	Time  0.018 ( 0.024)	Data  0.006 ( 0.006)	Loss nan (nan)	Acc@1  12.50 ( 10.01)	Acc@5  56.25 ( 50.64)
Epoch: [0][400/938]	Time  0.020 ( 0.047)	Data  0.007 ( 0.008)	Loss nan (nan)	Acc@1  10.94 ( 10.05)	Acc@5  56.25 ( 50.60)
Epoch: [0][500/938]	Time  0.022 ( 0.043)	Data  0.005 ( 0.008)	Loss nan (nan)	Acc@1  12.50 ( 10.00)	Acc@5  43.75 ( 50.71)
Epoch: [0][600/938]	Time  0.020 ( 0.039)	Data  0.005 ( 0.007)	Loss nan (nan)	Acc@1   7.81 (  9.93)	Acc@5  56.25 ( 50.54)
Epoch: [0][700/938]	Time  0.018 ( 0.037)	Data  0.004 ( 0.007)	Loss nan (nan)	Acc@1   9.38 (  9.86)	Acc@5  46.88 ( 50.54)
Epoc

KeyboardInterrupt: 

In [33]:
from biotorch.training.functions import test

test(alex, loss, test_loader, device='cpu', top_k=1)

 * Acc@1 97.970 Acc@1 97.970


(tensor(97.9700), 64.33413543701172)

In [4]:
# create a CNN model using pytorch of 3 cnn layers with with 2 fully connected layers

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(2*2*128, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 2*2*128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
model = CNN()

# convert the CNN model to a BioModule model
model = BioModule(model, mode='dfa', output_dim=10)
print(model)

Module has been converted to dfa mode:

The layer configuration was:  {'type': 'dfa', 'options': {'constrain_weights': False, 'init': 'xavier', 'gradient_clip': False}}
- All the 3 <class 'torch.nn.modules.conv.Conv2d'> layers were converted successfully.
- All the 2 <class 'torch.nn.modules.linear.Linear'> layers were converted successfully.
BioModule(
  (module): CNN(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (fc1): Linear(in_features=512, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [10]:
from biotorch.training.trainer import Trainer

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss

adam = Adam(model.parameters(), lr=0.01)
lr_ = StepLR(adam, step_size=0.1)
loss = CrossEntropyLoss()

metric_config = {
    'top_k': 5,
    'display_iterations': 100,  # Display metrics every 100 iterations
    'layer_alignment': False,     # Enable layer-wise alignment
    'weight_ratio': False      # Set weight ratio for alignment regularization
}

In [16]:
trainer = Trainer(model=model, mode='dfa', loss_function=loss, optimizer=adam, train_dataloader=train_loader, val_dataloader=test_loader, device='cpu', epochs=100, output_dir='./output', lr_scheduler=lr_, metrics_config=metric_config)

In [20]:
trainer.run()

ValueError: Expected input batch_size (16) to match target batch_size (64).